In [1]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
%matplotlib inline
import os
from keras.preprocessing import image
import matplotlib.image as mpimg
from IPython.display import Image, display, HTML
import warnings

from keras.models import Model
from keras.layers import Input
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution1D, AveragePooling1D
from keras.layers.merge import concatenate
from keras import metrics
from sklearn.model_selection import LeaveOneOut,KFold

from keras import models
from keras import layers
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
from random import shuffle

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)

Using TensorFlow backend.


In [9]:
FullData = "datasets/extractedFeatures_dataset.csv"
posFile = "datasets/Positive.txt"
negFile = "datasets/Negative.txt"
x, y, f = readFeaturizedData(FullData, posFile, negFile)
dindex = list(range(0, len(y)))
shuffle(dindex)
x = x[dindex]
y = y[dindex]

/usr/common/software/python/2.7-anaconda-4.4/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (34,176,398,2376,10445) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [36]:
totalfeatures = len(f)
def supervisedModel():
    network = models.Sequential()
    network.add(layers.Dense(units=256, activation='relu', input_shape=(totalfeatures,)))
    #network.add(Dropout(0.2))
    network.add(layers.Dense(units=256, activation='relu'))
    #network.add(Dropout(0.2))
    network.add(layers.Dense(units=1, activation='sigmoid'))
    network.compile(loss='binary_crossentropy', 
                    optimizer='adam', 
                    metrics=['accuracy'])
    return network

In [37]:
def deepModel():
    dropout = 0.2
    ffgc_input = Input(shape=(41,4))
    ffgc_conv1 = Convolution1D(256, 2, name='g_conv1')(ffgc_input)
    ffgc_act1 = Activation('relu', name='g_activation1')(ffgc_conv1)
    ffgc_pool1 = AveragePooling1D(2)(ffgc_act1)
    ffgc_drop1 = Dropout(dropout)(ffgc_pool1)

    ffgc_conv2 = Convolution1D(256, 2, name='g_conv2')(ffgc_drop1)
    ffgc_act2 = Activation('relu', name='g_activation2')(ffgc_conv2)
    ffgc_pool2 = AveragePooling1D(2)(ffgc_act2)
    ffgc_drop2 = Dropout(dropout)(ffgc_pool2)
    ffgc_flat1 = Flatten()(ffgc_drop2)

    ffgc_dense1 = Dense(256, activation='relu', name='g_dense1', trainable=False)(ffgc_flat1)
    ffgc_drop3 = Dropout(dropout)(ffgc_dense1)
    ffgc_dense2 = Dense(128, activation='relu', name='g_dense2', trainable=False)(ffgc_drop3)
    ffgc_drop4 = Dropout(dropout)(ffgc_dense2)
    ffgc_dense3 = Dense(256, activation='relu', name='g_dense3', trainable=False)(ffgc_drop4)
    ffgc_drop5 = Dropout(dropout)(ffgc_dense3)
    ffgc_out = Dense(256, activation='relu', name='g_dense4', trainable=False)(ffgc_drop5)
    predictions = Dense(1, name='predictions', trainable=False)(ffgc_out)
    model = Model([ffgc_input], predictions)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [38]:
def readFeaturizedData(fullData, posData, negData):
    fulldata = pd.read_csv(fullData, sep=',')
    featurenames = list(fulldata.columns.values)
    X = np.array(fulldata)
    dY1 = np.array(pd.read_csv(posData, sep=','))
    Y1 = np.full(len(dY1) + 1, 1)
    dY2 = np.array(pd.read_csv(negData, sep=','))
    Y2 = np.full(len(dY2) + 1, 0)
    Y = np.concatenate((Y1, Y2), axis = 0)
    return X, Y, featurenames

In [39]:
def readSequences(filename):
    sequences = []
    seqfile = open(filename)
    for line in seqfile.readlines():
        sequences.append(line.strip())
    return sequences

In [40]:
def onehotcoding(sequences):
    alphabetdict = {'A': [1, 0, 0, 0],
                    'C': [0, 1, 0, 0],
                    'G': [0, 0, 1, 0],
                    'T': [0, 0, 0, 1]}
    sequencelist = []
    for seq in sequences:
        seq_array = []
        for alphabet in seq:
            seq_array.append(alphabetdict[alphabet.upper()])
        seq_array = np.array(seq_array)
        sequencelist.append(seq_array)
    sequencelist = np.array(sequencelist)
    return sequencelist

In [41]:
positiveSeqs = readSequences(posFile)
negativeSeqs = readSequences(negFile)
posCodings = onehotcoding(positiveSeqs)
negCodings = onehotcoding(negativeSeqs)
allsequences = np.concatenate((posCodings, negCodings))

In [42]:
features, target = make_classification(n_samples = len(x),
                                       n_features = totalfeatures,
                                       n_informative = 350,
                                       n_redundant = 0,
                                       n_classes = 2,
                                       weights = [.5, .5],
                                       random_state = 0)

In [43]:
neural_network = KerasClassifier(build_fn=supervisedModel, 
                                 epochs=150, 
                                 batch_size=128, 
                                 verbose=0)

In [44]:
score = cross_val_score(neural_network, features, target, cv=10)
print("Accuracy:", score.mean())

('Accuracy:', 0.80965908874164927)


In [30]:
#loo = LeaveOneOut()
#n = 0
#accuracy = 0
#for train, test in loo.split(x):
#    dmodel = supervisedModel()
#    dmodel.fit(x[train], y[train], batch_size=128, epochs=10, verbose=0)
#    score, acc = dmodel.evaluate(x[test], y[test])
#    accuracy += float(acc)
#    print n,":", acc
#    n += 1
#accuracy = accuracy / n
#print("LOO accuracy:", accuracy)

In [45]:
kf = KFold(n_splits=10, shuffle=False, random_state=0)
cvscores = []
for train, test in kf.split(y):
    dmodel = deepModel()
    dmodel.fit(allsequences[train], y[train], batch_size = 100, epochs = 200, verbose = 0)
    score = dmodel.evaluate(allsequences[test], y[test], verbose=0)
    print("%s: %.2f%%" % (dmodel.metrics_names[1], score[1]*100))
    cvscores.append(score[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

acc: 52.27%
acc: 47.73%
acc: 49.43%
acc: 44.89%
acc: 44.32%
acc: 50.00%
acc: 46.59%
acc: 51.14%
acc: 48.30%
acc: 49.43%
48.41% (+/- 2.45%)
